#  probar una linea queno encuentre

# comparar performance

In [1]:
import pandas as pd
import numpy as np
import yaml

from sqlalchemy import create_engine
import psycopg2

import geopandas as gpd
from h3 import h3
from shapely.geometry import LineString, Point

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def imputar_detino_por_tarjeta(tramos_tarjeta,tolerancia_hex = 21):
    try:
        for i in range(len(tramos_tarjeta)):
            # tomo un tramo
            tramo = tramos_tarjeta.iloc[i]
            # tomo la linea del tramo
            linea = tramo.linea
            # y sus posibles paradas
            paradas_destino_posibles = paradas.loc[paradas.LINEA == linea,'h3_res_%i'%resolucion]

            # tomo las coordenadas del siguiente origen
            if i < (len(tramos_tarjeta)-1):
                #cuando hay un O_2
                O_2 = tramos_tarjeta.iloc[i+1]['h3_o']
            else:
                #si no tomar el primero del dia
                O_2 = tramos_tarjeta.iloc[0]['h3_o']
                
            #calculo la distancias        
            distancias_a_paradas = paradas_destino_posibles.map(
                lambda h : h3.h3_distance(
                    h3_address_origin = O_2,
                    h3_address_h3 = h))
            
            #evaluo tolerancia
            if any(distancias_a_paradas < tolerancia_hex):
                parada_destino = paradas_destino_posibles.loc[distancias_a_paradas.idxmin()]
            else:
                parada_destino = None 
                
            #asigno el destino
            tramos_tarjeta.loc[tramo.name,'h3_d'] = parada_destino
            
    except: 
        print('tarjeta:',tramos_tarjeta.tarjeta.unique())
        
    return tramos_tarjeta

def crear_Punto(row):
    if row.lat_d == None:
        return None
    else:
        return Point(row.lon_d,row.lat_d)
        

In [3]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [4]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [6]:
# traer trx de tarjetas que hayan pasado por la B
q = """
select *
from tramos_linea_b
limit 70000;
"""
tramos = pd.read_sql(q, conn)
tramos['delta'] = pd.to_timedelta(tramos['delta'])
tramos.head()

,tarjeta,tramo_id,linea,fecha,lat_o,lon_o,delta,h3_o,lat_d,lon_d,h3_d
0,2482584,0,SUBTE,2018-10-10 09:00:47,-34.604581,-58.405399,00:53:34,8bc2e311e96bfff,None,None,None
1,2482584,1,SUBTE,2018-10-10 09:54:21,-34.607802,-58.373956,04:11:55,8bc2e3113736fff,None,None,None
2,2482584,2,LINEA D,2018-10-10 14:06:16,-34.599757,-58.397924,04:10:46,8bc2e311e119fff,None,None,None
3,2482584,3,LINEA B,2018-10-10 18:17:02,-34.603297,-58.375072,NaT,8bc2e311360afff,None,None,None
4,2482609,0,SUBTE,2018-10-10 13:50:57,-34.608935,-58.406036,05:19:37,8bc2e3111272fff,None,None,None


In [7]:
len(tramos.tarjeta.unique())

18262

In [8]:
resolucion = 11
tolerancia_metros = 1000


In [9]:
distancia_entre_hex = h3.edge_length(resolution=resolucion, unit='m') * 2 
tolerancia_hex = np.ceil(tolerancia_metros / distancia_entre_hex)
tolerancia_hex

21.0

In [10]:
q = """
select p."LINEA", p."LATITUDE",p."LONGITUDE",p."h3_res_%i"
from paradas p
"""%resolucion
paradas = pd.read_sql(q, conn)
paradas.head()

,LINEA,LATITUDE,LONGITUDE,h3_res_11
0,LINEA 501,-34.887373,-58.384068,8bc2e39945adfff
1,LINEA 501,-34.887670,-58.387713,8bc2e3994cdbfff
2,LINEA 501,-34.887542,-58.390125,8bc2e39941b3fff
3,LINEA 501,-34.890546,-58.390244,8bc2e3994c76fff
4,LINEA 501,-34.892161,-58.391970,8bc2e3994d45fff


In [ ]:
%time destinos = tramos.groupby(['tarjeta']).apply(imputar_detino_por_tarjeta,tolerancia_hex=tolerancia_hex)
destinos

In [ ]:
tramos_tarjeta = tramos.loc[tramos.tarjeta ==32,:]
tramos_tarjeta

In [ ]:
imputar_detino_por_tarjeta(tramos_tarjeta)

In [ ]:
destinos

In [ ]:
for i in range(len(tramos_tarjeta)):
    print(i)
    # tomo un tramo
    tramo = tramos_tarjeta.iloc[i]
    # tomo la linea del tramo
    linea = tramo.linea
    # y sus posibles paradas
    paradas_destino_posibles = paradas.loc[paradas.LINEA == linea,'h3_res_%i'%resolucion]
    #print(paradas_destino_posibles)
    # tomo las coordenadas del siguiente origen
    if i < (len(tramos_tarjeta)-1):
        #cuando hay un O_2
        O_2 = tramos_tarjeta.iloc[i+1]['h3_o']
    else:
        #si no tomar el primero del dia
        O_2 = tramos_tarjeta.iloc[0]['h3_o']
    #calculo la distancias        
    distancias_a_paradas = paradas_destino_posibles.map(
        lambda h : h3.h3_distance(
            h3_address_origin = O_2,
            h3_address_h3 = h))
    
    if any(distancias_a_paradas < tolerancia_hex):
        parada_destino = paradas_destino_posibles.loc[distancias_a_paradas.idxmin()]
    else:
        parada_destino = None 
    print(parada_destino)

In [ ]:
O_2

In [ ]:
        for i in range(len(tramos_tarjeta)):

            tramo = tramos_tarjeta.iloc[i]
            check_equivalencias = equivalencias[tramo.linea]
            if not isinstance(check_equivalencias,list):
                check_equivalencias = [check_equivalencias]

            linea = rec.loc[rec.linea.isin(check_equivalencias),:]
            if i < (len(tramos_tarjeta)-1):
                #cuando hay un O_2
                O_2 = tramos_tarjeta.iloc[i+1]['geometry']
            else:
                #si no tomar el primero del dia
                O_2 = tramos_tarjeta.geometry.iloc[0]

            # ramal mas cercano al origen siguiente
            distancias = linea.geometry.distance(O_2)

            if any(distancias < tolerancia_metros):
                ramal_O_1 = linea.loc[distancias.idxmin(),'geometry']

                D_1 = ramal_O_1.interpolate(ramal_O_1.project(O_2))
                tramos_tarjeta.loc[tramo.name,'lat_d'] = D_1.y
                tramos_tarjeta.loc[tramo.name,'lon_d'] = D_1.x

            else:
                tramos_tarjeta.loc[tramo.name,'lat_d'] = None
                tramos_tarjeta.loc[tramo.name,'lon_d'] = None
    except: 
        print('tarjeta:',tramos_tarjeta.tarjeta.unique())
        print('dia',tramos_tarjeta.dia.unique())

        

In [ ]:
#tarjetas
#ejemplo 209215
#

In [ ]:
with open('../carto/recorridos_equivalencias.yaml') as file:
    equivalencias = yaml.load(file, Loader=yaml.FullLoader)
    
rec = gpd.read_file('../carto/recorridos.geojson')
tramos = gpd.read_file('../data/tramos.geojson')

In [ ]:
#elimino las tarjetas que hayan viajado en alguna de las lineas con problemas
#lineas_con_problemas = [ 'LINEA 157 SFE', 'LINEA_906', 'STAFE_SANTA FE_LINEA_907', 'LINEA_C_SFE','LINEA_013_SFE']
#tarjetas_eliminar = tramos.loc[tramos.linea.isin(lineas_con_problemas),'tarjeta'].unique()
#tramos = tramos.loc[~tramos.tarjeta.isin(tarjetas_eliminar),:].copy()

In [ ]:
tramos['lat_d'] = None
tramos['lon_d'] = None

In [ ]:
tolerancia_metros = 1000

In [ ]:
#tarjeta = 209215
#tarjeta = 437
#tarjeta = 747
tarjeta = 1133
tramos_tarjeta = tramos.loc[tramos.tarjeta == tarjeta,:].copy()

In [ ]:
tramos_tarjeta.groupby(['tarjeta','dia']).apply(imputar_detino_por_tarjeta_dia)

In [ ]:
%time destinos = tramos.groupby(['tarjeta','dia']).apply(imputar_detino_por_tarjeta_dia)


In [ ]:
destinos.lat_d.isnull().sum() /len(destinos) *100

In [ ]:
destinos.lon_d.isnull().sum() /len(destinos) *100

In [ ]:
destinos['lat_o'] = destinos.geometry.y
destinos['lon_o'] = destinos.geometry.x

In [ ]:
destinos.drop(['geometry'],axis=1).to_csv('../data/tabla_tramos_completos_epsg3857.csv',index=False)

In [ ]:
destinos.head()

In [ ]:
destinos_geoserie = gpd.GeoDataFrame(geometry = destinos.apply(crear_Punto,axis=1),crs='EPSG:3857')
destinos_geoserie['4326']=destinos_geoserie[destinos_geoserie.geometry.notnull()].to_crs('EPSG:4326')
destinos_geoserie = gpd.GeoSeries(destinos_geoserie['4326'])
destinos_geoserie.head()

In [ ]:
destinos = destinos.to_crs('EPSG:4326')

In [ ]:
destinos['lat_o'] = destinos.geometry.y
destinos['lon_o'] = destinos.geometry.x
destinos['lat_d'] = destinos_geoserie.y
destinos['lon_d'] = destinos_geoserie.x

In [ ]:
destinos.head()

In [ ]:
destinos.drop(['geometry'],axis=1).to_csv('../data/tabla_tramos_completos_epsg4326.csv',index=False)